In [331]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as ss
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import os

data_dir = ""
for dirname, _, filenames in os.walk('../../data/'):
    data_dir = dirname
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


../../data/tcdml1920-rec-click-pred--submission file (example).csv
../../data/tcdml1920-rec-click-pred--submission file.csv
../../data/tcdml1920-rec-click-pred--test.csv
../../data/tcdml1920-rec-click-pred--training.csv


In [169]:
raw_train_df = pd.read_csv(data_dir+"/tcdml1920-rec-click-pred--training.csv")
raw_test_df = pd.read_csv(data_dir+"/tcdml1920-rec-click-pred--test.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,5,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [360]:
def data_cleaning(train_df, test_df, split_method='time_viewed'):
    
    time_cols = ['query_word_count', 'query_detected_language', 'query_document_id', 'item_type', 'application_type', 
            'abstract_char_count', 'abstract_detected_language', 
            'app_lang', 'country_by_ip', 'local_hour_of_request', 'hour_request_received',
            'algorithm_class','recommendation_algorithm_id_used', 'cbf_parser', 'search_title', 'search_keywords',
            'search_abstract']    
    
   
    jabref_cols = ['query_char_count', 'app_lang', 'app_version', 'country_by_ip', 'timezone_by_ip', 
               'local_hour_of_request', 'algorithm_class', 'recommendation_algorithm_id_used']

    cbf_cols = ['cbf_parser']

    com_blog_cols = ['query_word_count', 'query_detected_language', 'abstract_char_count',
                     'application_type', 'hour_request_received', 'item_type',
                     'country_by_ip', 'algorithm_class', 'recommendation_algorithm_id_used']
    
    id_col = ['recommendation_set_id']
    
    dep_cols = ['clicks', 'ctr', 'set_clicked']
    
    train_df = train_df[train_df['rec_processing_time']<40]
    train_df = train_df[train_df['clicks']<= train_df['number_of_recs_in_set']]
    
    train_df = train_df[~(train_df['app_version'].isna())]
    
    cleaned_data = {}
    
    for key, df in {'test':test_df, 'train':train_df}.items():
        print(key)
        df = df.replace("\\N", np.nan)
        cols_to_change = ['query_word_count', 'query_char_count', 'local_hour_of_request', 'recommendation_algorithm_id_used',
                     'abstract_char_count', 'abstract_word_count']
        df[cols_to_change] = df[cols_to_change].astype('float64')
        df['q_doc_id_provided'] = df['query_document_id'].apply(lambda x: 0 if pd.isnull(x) else 1)
        
        data_dict = {}
        if split_method=='app-cbf':
            
            if key=='test':
                com_blog_cols = com_blog_cols + id_col
                jabref_cols = jabref_cols + id_col
            else:
                com_blog_cols = list(set(com_blog_cols + ['set_clicked']) - set(id_col))
                jabref_cols = list(set(jabref_cols + ['set_clicked']) - set(id_col))
                
            cblog_all = df[(df['application_type']=='blog') | (df['application_type']=='e-commerce') | (df['application_type']=='0')][com_blog_cols+cbf_cols]
        
            cblog_cbf = cblog_all[cblog_all['algorithm_class']=='content_based_filtering']
            #cblog_cbf = cblog_cbf[~(cblog_cbf['country_by_ip'].isna())]
            cblog_other = df[(df['application_type']=='blog') | (df['application_type']=='e-commerce') | (df['application_type']=='0')][com_blog_cols]
            cblog_other = cblog_other[cblog_other['algorithm_class']!='content_based_filtering']
#             cblog_other = cblog_other[~(cblog_other['country_by_ip'].isna())]
            
            dig_all = df[df['application_type']=='digital_library']
            dig_cbf = dig_all[dig_all['algorithm_class']=='content_based_filtering'][jabref_cols+cbf_cols+ ['query_detected_language', 'q_doc_id_provided']]
            #dig_cbf = dig_cbf[~((dig_cbf['country_by_ip'].isna()) | (dig_cbf['app_lang'].isna()) | (dig_cbf['app_version'].isna()))]
            dig_other = df[df['application_type']=='digital_library'][jabref_cols]
            dig_other = dig_other[dig_other['algorithm_class']!='content_based_filtering']
            #dig_other = dig_other[~((dig_other['app_lang'].isna()) | (dig_other['app_version'].isna()))]
            data_dict['cblog_cbf'] = cblog_cbf
            data_dict['cblog_other'] = cblog_other
            data_dict['dig_cbf'] = dig_cbf
            data_dict['dig_other'] = dig_other
            cleaned_data[key] = data_dict
            
        elif split_method=='time_viewed':
            if key=='test':
                df = df[~(df['application_type'].isna())]
                cleaned_data[key] = df
            else:
                df = df[(~df['time_recs_viewed'].isna()) | (df['set_clicked']==1)]
                df = df[time_cols + ['set_clicked']]
                cleaned_data[key] = df
        elif split_method=='algo':
            pass        
        
    return cleaned_data

In [361]:
cat_cols = ['item_type', 'application_type', 'app_lang', 'app_version', 'country_by_ip', 'timezone_by_ip', 
            'algorithm_class','recommendation_algorithm_id_used',
            'cbf_parser', 'query_detected_language', 'abstract_detected_language']

cleaned_data = data_cleaning(raw_train_df, raw_test_df, 'time_viewed')

test
train


In [343]:
testlen = 0
for key, data in cleaned_data['test'].items():
    testlen = testlen + data.shape[0]

print(testlen)

9145


In [218]:
raw_test_df[raw_test_df['country_by_ip'].isin(country_index)]['country_by_ip'].value_counts()

DE    2653
US    2207
CN     353
BR     283
GB     275
FR     203
CA     197
AT     188
AU     183
IN     162
IT     159
IE     157
CH     145
ES     135
NL     127
RU     100
JP      96
BE      93
MX      85
PL      84
CO      64
PT      56
DK      54
\N      51
NO      51
ID      51
SE      49
PK      42
ZA      42
TR      38
EC      35
GR      34
CZ      34
SG      32
NZ      31
FI      29
KR      29
TW      28
HK      27
PH      22
CL      21
DZ      21
MY      19
TH      19
PE      19
IL      19
AR      18
HU      17
RO      15
IR      14
KE      14
VN      14
LT      12
SA      12
RS      12
EE      11
EG      10
LV      10
HR       9
TN       8
LU       8
BD       7
UA       7
GH       7
SK       7
CR       7
TT       6
SI       6
BG       6
AE       6
BA       6
NG       5
UY       5
BO       4
MU       4
LK       4
BY       4
MA       4
BJ       3
IS       3
PR       3
PY       3
PS       2
TZ       2
NP       2
UG       2
KZ       2
CM       2
ET       2
BH       2
GT       2

In [351]:
for key, dataset in cleaned_data['train'].items():
    print('\nDataset: ', key)
    for col in dataset.columns:
        if np.nan in dataset[col].value_counts(dropna=False).index:
            print('Col: ', col)
            num = dataset[col].value_counts(dropna=False).loc[np.nan]
            percent = dataset[col].value_counts(dropna=False, normalize= True).loc[np.nan]
            print('Num: ', num, ' Percent: ', percent)


Dataset:  cblog_cbf

Dataset:  cblog_other

Dataset:  dig_cbf

Dataset:  dig_other


In [363]:
for col in cleaned_data['train'].columns:
        if np.nan in cleaned_data['train'][col].value_counts(dropna=False).index:
            print('Col: ', col)
            num = cleaned_data['train'][col].value_counts(dropna=False).loc[np.nan]
            percent = cleaned_data['train'][col].value_counts(dropna=False, normalize= True).loc[np.nan]
            print('Num: ', num, ' Percent: ', percent)

Col:  query_word_count
Num:  7  Percent:  0.0003633532312483779
Col:  query_detected_language
Num:  37  Percent:  0.0019205813651699973
Col:  query_document_id
Num:  4112  Percent:  0.21344406955618997
Col:  item_type
Num:  2116  Percent:  0.10983649104593823
Col:  abstract_char_count
Num:  4584  Percent:  0.23794445886322346
Col:  abstract_detected_language
Num:  4585  Percent:  0.23799636646768751
Col:  app_lang
Num:  4477  Percent:  0.2323903451855697
Col:  country_by_ip
Num:  142  Percent:  0.0073708798338956655
Col:  local_hour_of_request
Num:  1787  Percent:  0.09275888917726446
Col:  recommendation_algorithm_id_used
Num:  277  Percent:  0.014378406436542954
Col:  cbf_parser
Num:  6230  Percent:  0.32338437581105633


In [350]:
for dataset, data in cleaned_data.items():
    for key, df in data.items():
        
        if key=='cblog_cbf':
            missing_values = {'query_detected_language': df['query_detected_language'].mode()[0], 
                              'country_by_ip': 'missing', 'abstract_char_count': df['abstract_char_count'].median(), 
                            'item_type': 'missing', 'query_word_count': df['query_word_count'].median()}
            cleaned_data[dataset][key] = df.fillna(value=missing_values)
            
        elif key=='cblog_other':    
            missing_values = {'query_detected_language': df['query_detected_language'].mode()[0],
                              'abstract_char_count': df['abstract_char_count'].median(), 'query_word_count': df['query_word_count'].median(),
                         'item_type': 'missing', 'country_by_ip': 'missing'}
            cleaned_data[dataset][key] = df.fillna(value=missing_values)
           
        elif key=='dig_cbf':
            missing_values = {'app_lang': df['app_lang'].mode()[0], 'country_by_ip': 'missing', 'timezone_by_ip': 'missing', 'query_detected_language': df['query_detected_language'].mode()[0], 
                         'local_hour_of_request': df['local_hour_of_request'].median()}
            cleaned_data[dataset][key]= df.fillna(value=missing_values)
            
        else:
            missing_values = {'app_lang': df['app_lang'].mode()[0], 'country_by_ip': 'missing', 'timezone_by_ip': 'missing', 
                'local_hour_of_request': df['local_hour_of_request'].median(), 'recommendation_algorithm_id_used': 33}
            cleaned_data[dataset][key]= df.fillna(value=missing_values)
            
       


In [345]:
def preprocess_data(cleaned_data, cat_cols, target, scaling_method):
    
    processed_data = {}
    
#     for key, df in cleaned_data['train']:
#         train_df = cleaned_data['train'][key]
#         test_df = cleaned_data['test'][key]
        
        
    for dataset, data in cleaned_data.items():
        processed_data[dataset] = {}
        for key, df in data.items():
            
            print('Type: ', dataset, ' Dataset: ', key, ' Shape: ', df.shape)
            df_cat_cols = list(set(cat_cols).intersection(set(df.columns)))
            
            if dataset=='train':
                df_num_cols = list(set(df.columns) - set(df_cat_cols) -set([target]))
            else:
                df_num_cols = list(set(df.columns) - set(df_cat_cols) - set(['recommendation_set_id']))
                
            if scaling_method=='standard':
                scaler = StandardScaler()
                df[df_num_cols] = scaler.fit_transform(df[df_num_cols])
                
            else:
                scaler = MinMaxScaler()
                df[df_num_cols] = scaler.fit_transform(df[df_num_cols])
            
            for col in df_cat_cols:
                df_x = cleaned_data['train'][key][[col, 'set_clicked']].astype({col: str}).groupby(by=[col]).mean()           
                df[col] = df[col].apply(lambda x: df_x.loc[str(x)][0] if str(x) in df_x.index else 0)   
            
        
            if dataset=='train': 
                X_train, X_valid, y_train, y_valid = train_test_split(df[df_num_cols+df_cat_cols].values, df[target].values, test_size=0.3)
                processed_data[dataset][key] = {'X': df[df_num_cols+df_cat_cols].values, 'y': df[target].values,
                                                'X_train': X_train, 'X_valid': X_valid, 'y_train': y_train, 'y_valid': y_valid}
            else:
                processed_data[dataset][key] = {'data': df[df_num_cols + df_cat_cols], 'id': df['recommendation_set_id']}
                
    return processed_data

In [352]:
processed_data = preprocess_data(cleaned_data, cat_cols, 'set_clicked', 'standard')

Type:  test  Dataset:  cblog_cbf  Shape:  (2663, 11)
Type:  test  Dataset:  cblog_other  Shape:  (1308, 10)
Type:  test  Dataset:  dig_cbf  Shape:  (3795, 12)
Type:  test  Dataset:  dig_other  Shape:  (1379, 9)
Type:  train  Dataset:  cblog_cbf  Shape:  (81623, 11)
Type:  train  Dataset:  cblog_other  Shape:  (33909, 10)
Type:  train  Dataset:  dig_cbf  Shape:  (193290, 12)
Type:  train  Dataset:  dig_other  Shape:  (76671, 9)


In [267]:
processed_data['train']['cblog_other']['X'].shape

(33830, 9)

In [277]:

results = {}
for key, data in processed_data['train'].items():
    logit = LogisticRegression()
    X_train, X_valid, y_train, y_valid = train_test_split(data['X'], data['y'], test_size=0.2)
    logit.fit(X_train, y_train)
    y_pred= logit.predict(X_valid)
    
    f1 = f1_score(y_valid, y_pred, average='micro')
    precision = precision_score(y_valid, y_pred, average='micro')
    recall = recall_score(y_valid, y_pred, average='micro')
    cm = confusion_matrix(y_valid, y_pred)
    results[key] = {'F1':f1, ' Precision': precision, 'Recall': recall, 'CM': cm}

In [353]:
test_preds = {}
#logit = LogisticRegression()
model = RandomForestClassifier(n_estimators=100)

for key, data in processed_data['test'].items():
    
    model.fit(processed_data['train'][key]['X'], processed_data['train'][key]['y'])
    y_pred= model.predict(data['data'])
    test_preds[key] = y_pred
    



In [354]:
result = np.vstack((np.hstack((processed_data['test']['cblog_cbf']['id'].values.reshape(-1, 1), test_preds['cblog_cbf'].reshape(-1, 1))),
                   np.hstack((processed_data['test']['cblog_other']['id'].values.reshape(-1, 1), test_preds['cblog_other'].reshape(-1, 1))),
                   np.hstack((processed_data['test']['dig_cbf']['id'].values.reshape(-1, 1), test_preds['dig_cbf'].reshape(-1, 1))),
                   np.hstack((processed_data['test']['dig_other']['id'].values.reshape(-1, 1), test_preds['dig_other'].reshape(-1, 1)))))
    
result = pd.DataFrame(data = result, columns=['recommendation_set_id', 'set_clicked'])
result['recommendation_set_id'] = result['recommendation_set_id'].astype(int)
result.to_csv('tcdml1920-rec-click-pred--submission file.csv', encoding='utf-8', index=False)

In [357]:
result['set_clicked'].sum()

43.0